# Clustering Algorithm

In [2]:
# importing libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# scaling 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# import PCA 
from sklearn.decomposition import PCA

# kmeans clustering 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples

import os

In [3]:
# CHECKING THE WORKING DIRECTORY
os.getcwd()

'C:\\Users\\User\\Desktop\\Github\\Untitled Folder'

In [7]:
# read pickle file as dataframe
dataset = pd.read_pickle('country_df.pkl')

In [8]:
# CHECKING THE ROWS AND COLUMNS OF THE DATASET
dataset.shape

(167, 10)

## Scaling
Why scale the data in this case?

the features have incomparable units (metrics are percentages, dollar values, whole numbers)
the range values of the features also vary (one for example is 0 to 200, and another 0 to 100,000)
this level of variance can negatively impact the performance of this model,as this model is based on measuring distances, it can do this by giving more weight to some features
by scaling we are removing potential bias that the model can have towards features with higher magnitudes

In [9]:
# columns argument ==> we'll use this later to create a new dataframe with the rescaled data 
columns = dataset.columns

In [10]:
#Scale the data: MinMaxScaler (normalised)

# the scaler to use will be 
scaler = MinMaxScaler()

# 'scaler' is for the rescaling technique, 'fit' function is to find the x_min and the x_max, 'transform' function applies formula to all elements of data
rescaled_dataset_minmax = scaler.fit_transform(dataset)
rescaled_dataset_minmax

ValueError: could not convert string to float: 'Afghanistan'

In [ ]:
#Scale the data: StandardScaler (standardised)
# in standardisation, all features will be transformed to have the properties of standard normal distribution with mean=0 and standard deviation=1
# the scaler to use will be 
scaler = StandardScaler()

# 'scaler' is for the rescaling technique, 'fit' function is to find the x_min and the x_max, 'transform' function applies formula to all elements of data
rescaled_dataset_standard = scaler.fit_transform(dataset)
rescaled_dataset_standard

In [ ]:
#Scaled dataframes
# minmax
# we need to create a new dataframe with the column lables and the rescaled values 
df_minmax = pd.DataFrame(data= rescaled_dataset_minmax , columns = columns )
df_minmax

In [ ]:
# standardisation
# we need to create a new dataframe with the column lables and the rescaled values 
df_standard = pd.DataFrame(data= rescaled_dataset_standard , columns = columns)
df_standard

## PCA: Principal Component Analysis
PCA with data scaled with StandardScaler

In [ ]:
# fit and transform
pca = PCA()
pca.fit(df_standard)
pca_data_standard = pca.transform(df_standard)

# percentage variation 
per_var = np.round(pca.explained_variance_ratio_*100, decimals =1)
labels = ['PC' + str(x) for x in range (1, len(per_var)+1)]

# plot the percentage of explained variance by principal component
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label = labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')
plt.show()

# plot pca
pca_df_standard = pd.DataFrame(pca_data_standard, columns = labels)

## PCA with data scaled with MinMaxScaler


In [ ]:
# fit and transform
pca = PCA()
pca.fit(df_minmax)
pca_data_minmax = pca.transform(df_minmax)

# percentage variation 
per_var = np.round(pca.explained_variance_ratio_*100, decimals =1)
labels = ['PC' + str(x) for x in range (1, len(per_var)+1)]

# plot the percentage of explained variance by principal component
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label = labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')
plt.show()

# plot pca
pca_df_minmax = pd.DataFrame(pca_data_minmax, columns = labels)

In [ ]:
# dataframe with PC1, PC2, P3, PC4
data2 = pca_df_standard.drop(['PC5','PC6','PC7','PC8','PC9'], axis = 1)
data2

### OBSERVATIONS

After doing PCA with both standardised and normalised versions of the original dataset, we can see that there are 4 principal components can explain about 90% of the distribution of the original data.

In [11]:
# Model: K-Means Clustering

km = KMeans (
    n_clusters = 3, # number of clusters/centroids to create
    init = 'random', # ‘random’: choose n_clusters observations (rows) at random from data for the initial centroids
    n_init = 10, # this is the default value. This is the number of times the k-means algorithm will be run with different centroid seeds
    max_iter = 300, # this is the default value. This is the maximum number of iterations of the k-means algorithm for a single run.
    tol = 1e-4, # this is the default value. This is the relative tolerance with regards to Frobenius norm of the difference in the cluster centers of two consecutive iterations to declare convergence.
    random_state = 0 # this is the default value. Determines random number generation for centroid initialization. Use an int to make the randomness deterministic.
)